<a href="https://colab.research.google.com/github/ggalarza1/stock_financials/blob/main/Stock_financials_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Stock Financial Data
How to put an Income statement in a table

In [1]:
from rich.table import Table
from rich.console import Console
import pandas as pd
import yfinance as yf
import textwrap

 # Fetch income statement data
income_statement = yf.Ticker("aapl").quarterly_financials

 # Convert to DataFrame and fill NaN with 0
income_df = income_statement.fillna(0)

 # Format numbers with commas
for column in income_df.select_dtypes(include=['number']):
     income_df[column] = income_df[column].apply('{:,}'.format)

#Changing the order of items. Placing Operating Income first.
operating_income_index = income_df.index[income_df.index == 'Operating Income'][0]

# Changing the new order to follow Operating Income first.
new_order = [operating_income_index] + list(income_df.index.difference([operating_income_index]))
reordered_income_df = income_df.reindex(new_order)

# Filter out the unwanted column --> Getting rid of 6/30/2023 and 9/30/2023
filtered_columns = [col for col in reordered_income_df.columns if col.strftime("%m/%d/%Y") not in ["06/30/2023", "09/30/2023"]]
filtered_df = reordered_income_df[filtered_columns]

# Create the table
table = Table(title="[bold]Income Statement (Millions)[bold]", show_lines=True, title_style="bold size=18")

# Add columns with custom formatting for "Metric"
table.add_column("Financials", justify="left", style="black", no_wrap=False, width=30)  # Set no_wrap=False
for date in filtered_df.columns:
    #Format date as day/month/year
    formatted_date = date.strftime("%m/%d/%Y")
    table.add_column(formatted_date, style="magenta", width=20)  # Convert Timestamp to string


# Iterate through rows of the DataFrame
for row in filtered_df.itertuples():
    # Wrap "Metric" text if longer than 30 characters
    wrapped_metric = textwrap.fill(row.Index, width=30)  # Wrap text to 30 characters

    # Prepare row data with wrapped metric
    row_data = [wrapped_metric]
    for date in filtered_df.columns:
        value = filtered_df.loc[row.Index, date]
        # Check if the current metric is one of the exceptions
        if row.Index in ["Interest Expense","Interest Income", "Interest Expense Non Operating", "Interest Income Non Operating","Tax Rate For Cals", "Basic EPS", "Diluted EPS", "Tax Effect Of Unusual Items","Net Non Operating Interest Income Expense", "Net Interest Income",]:
            row_data.append(value)  # Keep the original value
        else:
          try:
            # Convert value to numeric and divide by 1 million
            value = float(value.replace(",", "")) / 1_000_000
            # Format value with commas
            row_data.append("{:,.0f}".format(value))
          except ValueError:
              row_data.append(str(filtered_df.loc[row.Index, date]))

    # Add the row to the table
    table.add_row(*row_data)

# Print the table using Rich
console = Console()
console.print(table)

<ipython-input-1-a57ff0e1dd93>:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  income_df = income_statement.fillna(0)


Income Statement (Millions)                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Financials               ┃ 12/31/2024     ┃ 09/30/2024     ┃ 06/30/2024      ┃ 03/31/2024     ┃ 12/31/2023      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Operating Income         │ 42,832         │ 29,591         │ 25,352          │ 27,900         │ 40,373          │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ Basic Average Shares     │ 15,082         │ 15,172         │ 15,288          │ 15,406         │ 15,510          │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ Basic EPS                │ 2.41           │ 0.97           │ 1.4             │ 1.53           │ 2.19            │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ Cost Of Revenue          │ 66,025         │ 51,051         │ 46,099          │ 48,482         │ 64,720          │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ Diluted Average Shares   │ 15,151         │ 15,243         │ 15,348          │ 15,465         │ 15,577          │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ Diluted EPS              │ 2.4            │ 0.97           │ 1.4             │ 1.53           │ 2.18            │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ Diluted NI Availto Com   │ 36,330         │ 14,736         │ 21,448          │ 23,636         │ 33,916          │
│ Stockholders             │                │                │                 │                │                 │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ EBIT                     │ 42,832         │ 29,591         │ 25,352          │ 27,900         │ 40,373          │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ EBITDA                   │ 45,912         │ 32,502         │ 28,202          │ 30,736         │ 43,221          │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ Gross Profit             │ 58,275         │ 43,879         │ 39,678          │ 42,271         │ 54,855          │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ Interest Expense         │ 0.0            │ 0.0            │ 0.0             │ 0.0            │ 0.0             │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ Interest Expense Non     │ 0.0            │ 0.0            │ 0.0             │ 0.0            │ 0.0             │
│ Operating                │                │                │                 │                │                 │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ Interest Income          │ 0.0            │ 0.0            │ 0.0             │ 0.0            │ 0.0             │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ Interest Income Non      │ 0.0            │ 0.0            │ 0.0             │ 0.0            │ 0.0             │
│ Operating                │                │                │                 │                │                 │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼───

In [ ]:
from rich.table import Table
from rich.console import Console
import pandas as pd
import yfinance as yf
import textwrap

 # Fetch income statement data
balance_statement = yf.Ticker("aapl").balance_sheet

 # Convert to DataFrame and fill NaN with 0
balance_df = balance_statement.fillna(0)

 # Format numbers with commas
for column in balance_df.select_dtypes(include=['number']):
     balance_df[column] = balance_df[column].apply('{:,}'.format)

#Changing the order of items. Placing Operating Income first.
operating_income_index = balance_df.index[balance_df.index == 'Operating Income'][0]

# Changing the new order to follow Operating Income first.
new_order = [operating_income_index] + list(income_df.index.difference([operating_income_index]))
reordered_income_df = income_df.reindex(new_order)

# Filter out the unwanted column --> Getting rid of 6/30/2023 and 9/30/2023
filtered_columns = [col for col in reordered_income_df.columns if col.strftime("%m/%d/%Y") not in ["06/30/2023", "09/30/2023"]]
filtered_df = reordered_income_df[filtered_columns]

# Create the table
table = Table(title="[bold]Income Statement (Millions)[bold]", show_lines=True, title_style="bold size=18")

# Add columns with custom formatting for "Metric"
table.add_column("Financials", justify="left", style="black", no_wrap=False, width=30)  # Set no_wrap=False
for date in filtered_df.columns:
    #Format date as day/month/year
    formatted_date = date.strftime("%m/%d/%Y")
    table.add_column(formatted_date, style="magenta", width=20)  # Convert Timestamp to string


# Iterate through rows of the DataFrame
for row in filtered_df.itertuples():
    # Wrap "Metric" text if longer than 30 characters
    wrapped_metric = textwrap.fill(row.Index, width=30)  # Wrap text to 30 characters

    # Prepare row data with wrapped metric
    row_data = [wrapped_metric]
    for date in filtered_df.columns:
        value = filtered_df.loc[row.Index, date]
        # Check if the current metric is one of the exceptions
        if row.Index in ["Interest Expense","Interest Income", "Interest Expense Non Operating", "Interest Income Non Operating","Tax Rate For Cals", "Basic EPS", "Diluted EPS", "Tax Effect Of Unusual Items","Net Non Operating Interest Income Expense", "Net Interest Income",]:
            row_data.append(value)  # Keep the original value
        else:
          try:
            # Convert value to numeric and divide by 1 million
            value = float(value.replace(",", "")) / 1_000_000
            # Format value with commas
            row_data.append("{:,.0f}".format(value))
          except ValueError:
              row_data.append(str(filtered_df.loc[row.Index, date]))

    # Add the row to the table
    table.add_row(*row_data)

# Print the table using Rich
console = Console()
console.print(table)